# Plan observable transits tonight: 

In [125]:
%matplotlib inline
from astroplan import (FixedTarget, Observer, is_event_observable, 
                       AirmassConstraint, AtNightConstraint, 
                       MoonSeparationConstraint)
from astroplan.periodic import EclipsingSystem
import astropy.units as u
from astropy.time import Time
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astroquery.simbad import Simbad

Simbad.add_votable_fields('flux(V)')

Download a table of known transiting exoplanets: 

In [15]:
from astroquery.exoplanet_orbit_database import ExoplanetOrbitDatabase

exoplanet_table = ExoplanetOrbitDatabase.get_table()
transit_table = exoplanet_table[exoplanet_table['TRANSIT'] == 1]

Create an astroplan [`EclipsingSystem`](https://astroplan.readthedocs.io/en/latest/api/astroplan.EclipsingSystem.html) object for each transiting exoplanet: 

In [128]:
periods = transit_table['PER']
epochs = transit_table['T0']
durations = transit_table['T14']
names = transit_table['NAME']
simbad_names = transit_table['SIMBADNAME']
radius_ratio = transit_table['RR']
sky_coords = transit_table['sky_coord']

eclipsing_systems = []
depths = []
coords = []
for p, t0, dur, name, rprs, coord in zip(periods, epochs, durations, names, radius_ratio, sky_coords):
    if not any([isinstance(i, np.ma.core.MaskedConstant) for i in [p, t0, dur]]):
        eclisping_sys = EclipsingSystem(Time(t0, format='jd'), p, duration=dur, name=name)
        eclipsing_systems.append(eclisping_sys)
        depths.append(rprs**2)
        coords.append(coord)

Initialize the observatory, and say we're observing tonight: 

In [83]:
mro = Observer.at_site('MRO')
start, end = mro.tonight(Time.now() + 2*u.day)

Define observing constraints: 

In [95]:
constraints = [AirmassConstraint(max=3), AtNightConstraint.twilight_civil(), 
               MoonSeparationConstraint(min=20*u.deg)]

Only compute observablility for transits of exoplanets with $R_p/R_\star >$ `minimum_radius_ratio` :

In [174]:
minimum_radius_ratio = 0.1
n_days = 5

transit_events = []
labels = 'Name, Depth, Ingress, Egress, RA, Dec, V'.split(', ')

for day in range(n_days):
    transits_day = []
    for sb_name, name, es, coord, rr in zip(simbad_names, names, eclipsing_systems, coords, radius_ratio):
        if rr > minimum_radius_ratio:
            next_transit = es.next_primary_ingress_egress_time(start+day*u.day, n_eclipses=1)

            if next_transit[0, 1] < end+day*u.day and next_transit[0, 0] > start+day*u.day: 
                observable = True
                
                if observable: 
                    query = Simbad.query_object(sb_name)
                    if query is not None and 'FLUX_V' in query.colnames: 
                        vmag = query['FLUX_V'][0]
                    else: 
                        vmag = np.nan
                    transits_day.append([name, round(rr**2, 3), next_transit[0, 0].iso, next_transit[0, 1].iso, 
                                         coord.ra.deg, coord.dec.deg, vmag])
    if len(transits_day) > 0:
        transit_events.append(Table(rows=transits_day, names=labels))
    else: 
        transit_events.append(None)

/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/table/np_utils.py:191: UserWarning: Warning: converting a masked element to nan.
  array_list = [np.array(obj[..., i].tolist()) for i in range(nfields)]
/Users/bmmorris/anaconda/lib/python3.5/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '--': this identifier has an incorrect format for catalog: 	DM : Durchmusterung catalogue
  (error.line, error.msg))
/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/table/np_utils.py:191: UserWarning: Warning: converting a masked element to nan.
  array_list = [np.array(obj[..., i].tolist()) for i in range(nfields)]
/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/table/np_utils.py:191: UserWarning: Warning: converting a masked element to nan.
  array_list = [np.array(obj[..., i].tolist()) for i in range(nfields)]


In [176]:
for i, table in enumerate(transit_events):
    print('\n\nTransits on {0} UTC:'.format((start+i*u.day).datetime.date()))
    if table is not None: 
        table.sort('V')
        table.pprint(max_width=1000)
    else:
        print('None')



Transits on 2018-12-12 UTC:
    Name     Depth         Ingress                  Egress                 RA                Dec         V 
------------ ----- ----------------------- ----------------------- ------------------ ----------------- ---
Kepler-423 b 0.015 2018-12-12 02:44:12.173 2018-12-12 05:23:04.944 299.12745833396906 47.59383055580987 nan


Transits on 2018-12-13 UTC:
None


Transits on 2018-12-14 UTC:
    Name     Depth         Ingress                  Egress                 RA               Dec          V 
------------ ----- ----------------------- ----------------------- ----------------- ------------------ ---
Kepler-785 b 0.028 2018-12-14 10:32:00.705 2018-12-14 14:05:33.825 62.61604166825612 -45.89822222179836 nan
  WASP-104 b 0.015 2018-12-14 07:45:52.009 2018-12-14 09:48:23.207 284.8280833323796  44.60873055564033 nan


Transits on 2018-12-15 UTC:
    Name     Depth         Ingress                  Egress                 RA                Dec                 V     